<a href="https://colab.research.google.com/github/kalz2q/mycolabnotebooks/blob/master/learnelixir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# メモ

elixir を齧る。かじる。

今のイメージ $\quad$ erlang 上で、erlang は 並行処理のためのシステムで、その erlang 上で理想的な言語を作ろうとしたら、ruby + clojure みたいな言語になった。

Dave Thomas と まつもとゆきひろ が勧めているのだからいい言語なのだろう。


* https://elixirschool.com/ja/lessons/basics/control-structures/
* https://magazine.rubyist.net/articles/0054/0054-ElixirBook.
* https://dev.to/gumi/elixir-01--2585
* https://elixir-lang.org/getting-started/introduction.html

---
本を買った。

プログラミング elixir

dave thomas, 笹田耕一・鳥居雪訳、 ohmsha
programming elixir |> 1.6

を読む。



In [2]:
%%capture
!wget https://packages.erlang-solutions.com/erlang-solutions_2.0_all.deb && sudo dpkg -i erlang-solutions_2.0_all.deb
!sudo apt update
!sudo apt install elixir

In [ ]:
!elixir -v
!date

Erlang/OTP 24 [erts-12.2.1] [source] [64-bit] [smp:2:2] [ds:2:2:10] [async-threads:1] [jit]

Elixir 1.13.0 (compiled with Erlang/OTP 24)
Sat Mar  5 21:42:24 UTC 2022


---
メモ

`!elixir -h` (ヘルプ)としたらシェルワンライナー `elixir -e` が使えるらしいことがわかった。

`iex` というのがインタラクティブ環境なのだが、colab では使いにくいので `elixir -e` で代用する。

In [ ]:
!elixir -e 'IO.puts 3 + 3'
!elixir -e 'IO.puts "hello world!"'

6
hello world!


In [ ]:
# 次のようにすればファイルが作れる
%%writefile temp.exs
IO.puts "this is a pen."

Writing temp.exs


In [ ]:
# cat してみる
!cat temp.exs

IO.puts "this is a pen."

In [ ]:
# ファイルを elixir で実行する
!elixir temp.exs

this is a pen.


---
ネットで紹介されていた次のコードセルのコードはどうやって実行するのだろう。 今はわからなくていいと思うがとりあえず転記しておく。

説明:

このプログラムでは、Parallel というモジュールに pmap という関数を定義しています。 pmap は、与えられたコレクションに対して map（Ruby での Enumerable#map と同じようなものと考えて下さい）を行なうのですが、 各要素の処理を、要素数の分だけプロセスを生成し、各プロセスで並行に実行する、というものです。 ちょっと見ても、よくわからないような気がしますが、大丈夫、本書を読めば、わかるようになります。

とのこと。

In [ ]:
%%writefile temp.exs
defmodule Parallel do
  def pmap(collection, func) do
    collection
    |> Enum.map(&(Task.async(fn -> func.(&1) end)))
    |> Enum.map(&Task.await/1)
  end
end

result = Parallel.pmap 1..1000, &(&1 * &1)

IO.inspect result

Overwriting temp.exs


In [ ]:
!elixir temp.exs

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324,
 361, 400, 441, 484, 529, 576, 625, 676, 729, 784, 841, 900, 961, 1024, 1089,
 1156, 1225, 1296, 1369, 1444, 1521, 1600, 1681, 1764, 1849, 1936, 2025, 2116,
 2209, 2304, 2401, 2500, ...]


上の例で colab 環境で非同期処理が問題なく動くことが確認できたみたい。

---
次のもネットで紹介されていた例で、ハローワールド並行処理版


In [ ]:
%%writefile temp.exs
parent = self()
 
spawn_link(fn ->
  send parent, {:msg, "hello world"}
end)
 
receive do
  {:msg, contents} -> IO.puts contents
end

Overwriting temp.exs


In [ ]:
!elixir temp.exs

hello world


上の例でやっていることはつぎのような流れである。

1. spawn_linkという関数に渡された関数が、関数の内容を実行する。
2. 新しく作られたプロセス側では、メインプロセス側（parent）に “hello world” というメッセージを送る。
3. メインプロセス側は、どこからかメッセージが来ないかを待ち受けて（receive）、メッセージが来たらそれをコンソールに表示する。

In [ ]:
# 実験 とりあえず理解しない。 colab 環境でどうかだけ調べる。
%%writefile chain.exs
defmodule Chain do
  def counter(next_pid) do
    receive do
      n -> send next_pid, n + 1
    end
  end
 
  def create_processes(n) do
    last = Enum.reduce 1..n, self(),
      fn (_, send_to) -> spawn(Chain, :counter, [send_to]) end
    send last, 0
    receive do
      final_answer when is_integer(final_answer) ->
        "Result is #{inspect(final_answer)}"
    end
  end
 
  def run(n) do
    IO.puts inspect :timer.tc(Chain, :create_processes, [n])
  end
end

Writing chain.exs


In [ ]:
!elixir --erl "+P 1000000" -r chain.exs  -e "Chain.run(1_000_000)"

{4638957, "Result is 1000000"}


記事 https://ubiteku.oinker.me/2015/12/22/elixir試飲-2-カルチャーショックに戸惑う-並行指向プ/ のマシン Macbook Pro – 3 GHz Intel Core i7, 16GB RAM では 7 秒のところ、colab では 5 秒で終わってるね!!!!

手元のwindowsマシン intel core i5-9400 8gb ram でやったら次のようになった。  
{3492935, "Result is 1000000"}

あれ、速いじゃん!!!!

---
コメントは `#`

In [ ]:
%%writefile temp.exs
# コメント実験
str = "helloworld!!!!"
IO.puts str

Overwriting temp.exs


In [ ]:
!elixir temp.exs

helloworld!!!!


---
n 進数、整数 integer

In [ ]:
!elixir -e 'IO.puts 0b1111'
!elixir -e 'IO.puts 0o7777'
!elixir -e 'IO.puts 0xffff'
!elixir -e 'IO.puts 1000_000_00_0'

15
4095
65535
1000000000


整数型に上限下限 fixed limit はない。 factorial(10000) が計算できる。今はしない。


---
問題

10進数を $n$ 進数にベースを変えるのはどうするか。 python では `int()`, `bin()`, `oct()`, `hex()` があった。

In [ ]:
# python
print(0b1111)
print(0o7777)
print(0xffff)
print(int('7777',8))
print(bin(15))
print(oct(4095))
print(hex(65535))

15
4095
65535
4095
0b1111
0o7777
0xffff


In [ ]:
!elixir -e 'IO.puts 0b1111'
!elixir -e 'IO.puts 0o7777'
!elixir -e 'IO.puts 0xffff'
!echo
# Integer.to_string() と言う関数を使う
# <> はバイナリー連結
!elixir -e 'IO.puts "0b" <> Integer.to_string(15,2)'
!elixir -e 'IO.puts "0o" <> Integer.to_string(4095,8)'
!elixir -e 'IO.puts "0x" <> Integer.to_string(65535,16)'

15
4095
65535

0b1111
0o7777
0xFFFF


浮動小数点数 floating-point number

In [ ]:
!elixir -e 'IO.puts 1.532e-4'
# .0 とか 1. とかはエラーになる
!elixir -e 'IO.puts 98099098.0809898888'
!elixir -e 'IO.puts 0.00000000000000000000000001' #=> 1.0e-26
!elixir -e 'IO.puts 90000000000000000000000000000000000000000000000000000000'

1.532e-4
98099098.08098988
1.0e-26
999999999999999999999999999999999999999
90000000000000000000000000000000000000000000000000000000


文字列 string

string という型はない、みたい。

質問 型を調べる関数はあるか。type() とか。

質問 日本語についてはまた別途まとめて勉強する。

In [ ]:
!elixir -e 'IO.puts "日本語が書けますか"'
!elixir -e 'IO.puts "日本語が書けます"'

日本語が書けますか
日本語が書けます


In [ ]:
# 関数に括弧をつけることができる
# \ で escape できる
!elixir -e 'IO.puts (0b1111)'
!elixir -e 'IO.puts ("にほんご\n日本語")'
!elixir -e "IO.puts ('にほんご\n\"日本語\"')"

15
にほんご
日本語
にほんご
"日本語"


In [ ]:
# 文字連結 `+` ではない!!!!
!elixir -e 'IO.puts("ABCD"<>"EFGH")'

ABCDEFGH


`<>` と言う記号はバイナリ連結ということらしい。

---
値の埋め込み

`#{変数名}` を記述することで、変数の値を埋め込むことができる。


In [ ]:
!elixir -e 'val = 1000; IO.puts "val = #{val}"'

val = 1000


---
真偽値

elixir の 真偽値は true と false (小文字) で false と nil が false でそれ以外は true



In [ ]:
!elixir -e 'if true do IO.puts "true" end'
!elixir -e 'if True do IO.puts "true" end'
!elixir -e 'if False do IO.puts "true" end' # False が大文字なので
!elixir -e 'if false do IO.puts "true" else IO.puts "false" end'
!elixir -e 'if nil do IO.puts "true" else IO.puts "false" end'
!elixir -e 'if 0 do IO.puts "true" else IO.puts "false" end'
!elixir -e 'if (-1) do IO.puts "true" else IO.puts "false" end'
!elixir -e 'if [] do IO.puts "true" else IO.puts "false" end'
!elixir -e 'if "" do IO.puts "true" else IO.puts "false" end'

true
true
true
false
false
true
true
true
true


`null` はない。

# マッチ演算子 `=`

マッチ演算子 `=` はマッチ演算子である。 マッチ演算子を通して値を代入し、その後、マッチさせることができる。マッチすると、方程式の結果が返され、失敗すると、エラーになる。

In [3]:
!elixir -e 'IO.puts a = 1'
!elixir -e 'a =1; IO.puts 1 = a'
!elixir -e 'a =1; IO.puts 2 = a'

1
1
** (MatchError) no match of right hand side value: 1
    (stdlib 3.17) erl_eval.erl:450: :erl_eval.expr/5
    (stdlib 3.17) erl_eval.erl:893: :erl_eval.expr_list/6
    (stdlib 3.17) erl_eval.erl:408: :erl_eval.expr/5
    (elixir 1.13.0) lib/code.ex:404: Code.validated_eval_string/3


In [9]:
!elixir -e 'IO.inspect a = [1,2,3]' # リストは puts で表示できないので
!elixir -e '[a,b,c] = [1,2,3]; IO.puts c; IO.puts b'


[1, 2, 3]
3
2


上の例は、elixir は マッチ演算子 `=` があると左右がマッチするように最善を尽くす。 そのため、`[a,b,c] = [1,2,3]` で a,b,c に値が代入される。

In [ ]:
!elixir -e 'IO.inspect [1,2,[3,4,5]]'
!elixir -e '[a,b,c] = [1,2,[3,4,5]]; IO.inspect c; IO.inspect b'

[1, 2, [3, 4, 5]]
[3, 4, 5]
2


In [ ]:
# 実験 => エラー
!elixir -e 'IO.insepct [a,b] = [1,2,3]'

** (MatchError) no match of right hand side value: [1, 2, 3]
    (stdlib 3.15) erl_eval.erl:450: :erl_eval.expr/5
    (stdlib 3.15) erl_eval.erl:893: :erl_eval.expr_list/6
    (stdlib 3.15) erl_eval.erl:408: :erl_eval.expr/5
    (elixir 1.12.0) lib/code.ex:656: Code.eval_string_with_error_handling/3


In [ ]:
# 実験
!elixir -e 'IO.inspect a = [[1,2,3]]'
!elixir -e 'IO.inspect [a] = [[1,2,3]]'
!elixir -e '[a] = [[1,2,3]]; IO.inspect a'

[[1, 2, 3]]
[[1, 2, 3]]
[1, 2, 3]


In [ ]:
# 実験 => エラー
!elixir -e 'IO.insepct [a,b] = [a,b]'

  nofile:1

** (CompileError) nofile:1: undefined function a/0
    (stdlib 3.15) lists.erl:1358: :lists.mapfoldl/3


# いまここ

In [ ]:
# 実験
!elixir -e 'IO.puts a = :a'
!elixir -e 'a = :a; IO.inspect a = a'
!elixir -e 'a = :a; IO.puts a = a'
!elixir -e 'IO.puts :b'

a
:a
a
b


アンダースコア `_` で値を無視する。 ワルドカード。

なんでも受け付ける。

In [ ]:
!elixir -e 'IO.inspect [1,_,_]=[1,2,3]'

[1, 2, 3]


In [ ]:
!elixir -e 'IO.inspect [1,_,_]=[1,"cat","dog"]'

[1, "cat", "dog"]


変数は、バインド (束縛、紐付け) されると変更できない。

かと思ったらできてしまう。


In [ ]:
!elixir -e 'a = 1; IO.puts a = 2'

2


元の変数を指し示すピン演算子 (`^` カレット) がある。

In [ ]:
!elixir -e 'a = 1; IO.puts ^a = 2'

** (MatchError) no match of right hand side value: 2
    (stdlib 3.15) erl_eval.erl:450: :erl_eval.expr/5
    (stdlib 3.15) erl_eval.erl:893: :erl_eval.expr_list/6
    (stdlib 3.15) erl_eval.erl:408: :erl_eval.expr/5
    (elixir 1.12.0) lib/code.ex:656: Code.eval_string_with_error_handling/3


メモ $\quad$ 普通の関数型言語のように変数は変更できないルールにしてしまった方が簡単ではなかったか、と思わないでもない。 変数を不変にする、const 宣言みたいなのはないのか。

リストは不変 immutable なので安心。



In [ ]:
# 大文字にする capitalize
!elixir -e 'IO.puts name = String.capitalize "elixir"'

Elixir


In [ ]:
# 大文字にする upcase
!elixir -e 'IO.puts String.upcase "elixir"'

ELIXIR


# アトム

アトムは名前がそのまま値となる定数である。

**名前の前にコロン `:` をつけることでアトムになる。**

アトムの名前は utf-8 文字列 (記号を含む)、数字、アンダースコア `_` 、`@` で、終端文字としてのみ「!」や「?」が使える。

:fred $\quad$ :is_binary? $\quad$ ​:var@2 $\quad$ ​:<> $\quad$ ​:=== 

​:"func/3"​ $\quad$ ​​:"long john silver" $\quad$ ​:эликсир

​:mötley_crüe

メモ

In [ ]:
# 実験
!elixir -e 'IO.puts true === :true'
!elixir -e 'IO.puts :true'
!elixir -e 'IO.puts false === :false'

true
true
true


In [ ]:
# 実験 多分 colab の環境のせいで引用符が処理できない。2 バイト文字はオッケー。
!elixir -e 'IO.puts :fred'
!elixir -e 'IO.puts :is_binary?'
!elixir -e 'IO.puts :var@2'
!elixir -e 'IO.puts :<>'
!elixir -e 'IO.puts :==='
# !elixir -e 'IO.puts :"func/3"​'
# !elixir -e 'IO.puts ​​:"long john silver"'
!elixir -e 'IO.puts :эликсир'
!elixir -e 'IO.puts :mötley_crüe'
!elixir -e 'IO.puts :日本語はどうか'

fred
is_binary?
var@2
<>
===
эликсир
mötley_crüe
日本語はどうか


演算子

In [ ]:
!elixir -e 'IO.puts 1 + 2'
!elixir -e 'x = 10; IO.puts x + 1'
!elixir -e 'IO.puts 1 - 2'
!elixir -e 'x = 10; IO.puts x - 1'
!elixir -e 'IO.puts 5 * 2'
!elixir -e 'x = 10; IO.puts x * 4'
!echo 
!elixir -e 'IO.puts 5 / 2'
!elixir -e 'x = 10; IO.puts x / 3'

3
11
-1
9
10
40

2.5
3.3333333333333335


In [ ]:
# 浮動少数ではなく整数としての結果がほしい場合は div 関数を使用
!elixir -e 'IO.puts div(10,5)'
!elixir -e 'IO.puts div(10,4)'
# 割り算の余り、剰余を求める場合は rem関数を使用
!elixir -e 'IO.puts rem(10,4)'
!elixir -e 'IO.puts rem(10,3)'
!elixir -e 'IO.puts rem(10,2)'

2
2
2
1
0


In [ ]:
# 比較演算子
!elixir -e 'IO.puts 1 == 1'
!elixir -e 'IO.puts 1 != 1'
!elixir -e 'IO.puts ! (1 != 1)'
!elixir -e 'IO.puts 20.0 == 20'
!elixir -e 'IO.puts 20.0 === 20'
!elixir -e 'IO.puts 20.0 !== 20'

true
false
true
true
false
true


In [ ]:
# 論理演算子
# 論理和
!elixir -e 'IO.puts "ABC" == "ABC" || 20 == 30'
!elixir -e 'IO.puts "ABC" == "abc" || 20 == 30'
!echo
# 論理積
!elixir -e 'IO.puts "ABC" == "ABC" && 20 == 20'
!elixir -e 'IO.puts "ABC" == "ABC" && 20 == 30'
!elixir -e 'IO.puts "ABC" == "def" && 10 > 100'
!echo
# 否定
!elixir -e 'IO.puts !("ABC" == "ABC")'
!elixir -e 'IO.puts !("ABC" == "DEF")'

true
false

true
false
false

false
true


range

メモ $\quad$ range は型ではなく、struct である。 構造体?

`start..end` で表現される、とあるが、1..10 と書けばそれで range なのか?


In [ ]:
!elixir -e 'IO.inspect Enum.to_list(1..3)'
!elixir -e 'IO.inspect Enum.to_list(0..10//3)'
!elixir -e 'IO.inspect Enum.to_list(0..10//-3)'
!elixir -e 'IO.inspect Enum.to_list(10..0//-3)'
!elixir -e 'IO.inspect Enum.to_list(1..1)'
!elixir -e 'IO.inspect Enum.to_list(1..1//2)'
!elixir -e 'IO.inspect Enum.to_list(1..-1//2)'

[1, 2, 3]
[0, 3, 6, 9]
[]
[10, 7, 4, 1]
[1]
[1]
[]


In [ ]:
!elixir -e 'IO.inspect 1..9//2'

1..9//2


正規表現 regular expression

正規表現も型ではなく、struct である。

In [ ]:
!elixir -e 'IO.inspect Regex.run ~r{[aiueo]},"catapillar"'

["a"]


In [ ]:
!elixir -e 'IO.inspect Regex.scan ~r{[aiueo]},"catapillar"'

[["a"], ["a"], ["i"], ["a"]]


In [ ]:
!elixir -e 'IO.inspect Regex.split ~r{[aiueo]},"catapillar"'

["c", "t", "p", "ll", "r"]


In [ ]:
!elixir -e 'IO.inspect Regex.replace ~r{[aiueo]},"catapillar", "*"'

"c*t*p*ll*r"


# コレクション型

## タプル

タプルは波括弧 brace を用いて定義する。

タプルに限らず elixir のコレクションはすべて要素のタイプを限定しない。

通常 2 から 4 の要素であり、それ以上の要素数の場合、map や struct の利用を考える。

タプルは関数の返り値に便利に利用される。

パターンマッチングと組み合わせて使われる。


In [ ]:
!elixir -e 'IO.inspect {3.14, :pie, "Apple"}'

{3.14, :pie, "Apple"}


In [ ]:
!elixir -e '{status, count, action} = {3.14, :pie, "next"}; IO.puts action'

next


In [ ]:
!echo hello > temp.txt
!elixir -e '{status, file} = File.open("temp.txt"); IO.inspect {status, file}'
!elixir -e '{status, file} = File.read("temp.txt"); IO.inspect {status, file}'
!elixir -e '{status, file} = File.read("temp02.txt"); IO.inspect {status, file}'

{:ok, #PID<0.95.0>}
{:ok, "hello\n"}
{:error, :enoent}


In [ ]:
# 実験 タプルに ++ は使えるか。 => 使えない
# !elixir -e 'IO.inspect {3.14, :pie, "Apple"} ++ {3}'
# 実験 タプルに head は使えるか。 => 使えない
# !elixir -e 'IO.inspect hd {3.14, :pie, "Apple"}'
# 実験 タプルにパターンマッチングは使えるか。 => 使える
!elixir -e '{a,b,c} =  {3.14, :pie, "Apple"}; IO.inspect [c,a,b]'

/bin/bash: elixir: command not found


In [ ]:
# 実験
!elixir -e 'a=1; b=3; {b,a}={a,b}; IO.inspect {a,b}'

{3, 1}


## リスト

他の言語の配列 array と elixir のリストは違うので注意。 lisp のリストと似たような概念である。

カラのリストでなければ、head と tail がある。


In [ ]:
# リスト
!elixir -e 'IO.inspect [3.14, :pie, "Apple"]'


[3.14, :pie, "Apple"]


In [ ]:
# リスト先頭への追加(高速)
!elixir -e 'IO.inspect ["π" | [3.14, :pie, "Apple"]]'
# リスト末尾への追加(低速)
!elixir -e 'IO.inspect [3.14, :pie, "Apple"] ++ ["Cherry"]'

["π", 3.14, :pie, "Apple"]
[3.14, :pie, "Apple", "Cherry"]


上と下のコードセルでリストの連結を行っているが、++/2 演算子を用いている。 この `++/2` という表記は `++` が演算子自体で `/2` がアリティ (引数の数) を表す。 



---
質問 $\quad$ アリティとはなにか。

質問 $\quad$ リストの連結に `++` で文字列の連結 `<>` なのはなぜか。 オーバーライディングはあるのか。 文字列 string はリストではないのか。 長さを測る関数も別々なのか。

In [ ]:
# リストの連結
!elixir -e 'IO.inspect  [1, 2] ++ [3, 4, 1]'

** (ArgumentError) expected binary argument in <> operator but got: [1, 2]
    (elixir 1.12.0) lib/kernel.ex:1893: Kernel.wrap_concatenation/3
    (elixir 1.12.0) lib/kernel.ex:1880: Kernel.extract_concatenations/2
    (elixir 1.12.0) expanding macro: Kernel.<>/2
    nofile:1: (file)


In [ ]:
# リストの減算 
#  --/2 演算子は存在しない値を引いてしまってもオッケー
!elixir -e 'IO.inspect ["foo", :bar, 42] -- [42, "bar"]'
# 重複した値の場合、右辺の要素のそれぞれに対し、左辺の要素のうち初めて登場した同じ値が順次削除
!elixir -e 'IO.inspect [1,2,2,3,2,3] -- [1,2,3,2]'
# リストの減算の値のマッチには strict comparison が使われている
!elixir -e 'IO.inspect  [2] -- [2.0]'
!elixir -e 'IO.inspect  [2.0] -- [2.0]'

["foo", :bar]
[2, 3]
[2]
[]


In [ ]:
# head /tail
!elixir -e 'IO.inspect hd [3.14, :pie, "Apple"]'
!elixir -e 'IO.inspect tl [3.14, :pie, "Apple"]'

3.14
[:pie, "Apple"]


---
リストを頭部と尾部に分けるのに

* パターンマッチング
* cons 演算子( `|` )

を使うこともできる。

In [ ]:
!elixir -e '[head | tail] = [3.14, :pie, "Apple"]; IO.inspect head; IO.inspect tail'

3.14
[:pie, "Apple"]


## キーワードリスト

キーワードリストとマップは elixir の連想配列である。

キーワードリストは最初の要素がアトムのタプルからなる特別なリストで、リストと同様の性能になる。

In [ ]:
# キーワードリスト
!elixir -e 'IO.inspect [foo: "bar", hello: "world"]'
# タプルのリストとしても同じ
!elixir -e 'IO.inspect [{:foo, "bar"}, {:hello, "world"}]'
!elixir -e 'IO.inspect [foo: "bar", hello: "world"] == [{:foo, "bar"}, {:hello, "world"}]'

[foo: "bar", hello: "world"]
[foo: "bar", hello: "world"]
true


キーワードリストの 3 つの特徴

* キーはアトムである。
* キーは順序付けされている。
* キーの一意性は保証されない。

こうした理由から、キーワードリストは関数にオプションを渡すためによく用いられる。


In [ ]:
# 実験 リストの角括弧は省略できる
!elixir -e 'IO.inspect foo: "bar", hello: "world"'

[foo: "bar", hello: "world"]


In [ ]:
# 実験
!elixir -e 'IO.inspect [1, fred: 1, dave: 2]'
!elixir -e 'IO.inspect {1, fred: 1, dave: 2}'
!elixir -e 'IO.inspect {1, [{:fred,1},{:dave, 2}]}'

[1, {:fred, 1}, {:dave, 2}]
{1, [fred: 1, dave: 2]}
{1, [fred: 1, dave: 2]}


## マップ

* キーワードリストとは違ってどんな型のキーも使える。
* 順序付けされない。
* キーの一意性が保証されている。重複したキーが追加された場合は、前の値が置き換えられる。
* 変数をマップのキーにできる。
* `%{}` 構文で定義する。



In [ ]:
!elixir -e 'IO.inspect %{:foo => "bar", "hello" => :world}'
!elixir -e 'map = %{:foo => "bar", "hello" => :world}; IO.inspect map[:foo]'
!elixir -e 'map = %{:foo => "bar", "hello" => :world}; IO.inspect map["hello"]'
!echo
!elixir -e 'key = "hello"; IO.inspect %{key => "world"}'
!elixir -e 'IO.inspect %{:foo => "bar", :foo => "hello world"}'

%{:foo => "bar", "hello" => :world}
"bar"
:world

%{"hello" => "world"}
%{foo: "hello world"}


上下の例にあるように、アトムのキーだけを含んだマップには特別な構文がある。

In [ ]:
!elixir -e 'IO.inspect %{foo: "bar", hello: "world"} == %{:foo => "bar", :hello => "world"}'

true


In [ ]:
# 加えて、アトムのキーにアクセスするための特別な構文がある。
!elixir -e 'map = %{foo: "bar", hello: "world"}; IO.inspect map.hello'

"world"


In [ ]:
# マップの更新のための構文がある (新しい map が作成される)
# この構文は、マップに既に存在するキーを更新する場合にのみ機能する
!elixir -e 'map = %{foo: "bar", hello: "world"}; IO.inspect %{map | foo: "baz"}'

%{foo: "baz", hello: "world"}


In [ ]:
# 新しいキーを作成するには、`Map.put/3` を使用
!elixir -e 'map = %{hello: "world"}; IO.inspect  Map.put(map, :foo, "baz")'

%{foo: "baz", hello: "world"}


# いまここ 
# バイナリ binary

In [ ]:
# binaries
!elixir -e 'IO.inspect <<1,2>>'
!elixir -e 'IO.inspect <<1,10>>'
!elixir -e 'bin = <<1,10>>; IO.inspect byte_size bin'
!elixir -e 'bin = <<3::size(2),5::size(4),1::size(2)>>; IO.inspect bin'
!elixir -e 'IO.puts Integer.to_string(213,2)'
!elixir -e 'IO.puts 0b11'
!elixir -e 'IO.puts 0b0101'
!echo
!elixir -e 'bin = <<3::size(2),5::size(4),1::size(2)>>; IO.inspect byte_size bin'
!elixir -e 'bin = <<3::size(2),5::size(4),1::size(2)>>; IO.inspect :io.format("~-8.2b~n",:binary.bin_to_list(bin))'

<<1, 2>>
<<1, 10>>
2
<<213>>
11010101
3
5

1
11010101
:ok


# Date and Time 日付

In [ ]:
# Date and Time
!elixir -e 'IO.inspect Date.new(2021,6,2)'

{:ok, ~D[2021-06-02]}


In [ ]:
!elixir -e '{:ok, d1}=Date.new(2021,6,2); IO.inspect d1'
!elixir -e '{:ok, d1}=Date.new(2021,6,2); IO.inspect Date.day_of_week(d1)'
!elixir -e '{:ok, d1}=Date.new(2021,6,2); IO.inspect Date.add(d1,7)'
!elixir -e '{:ok, d1}=Date.new(2021,6,2); IO.inspect d1, structs: false'

~D[2021-06-02]
3
~D[2021-06-09]
%{__struct__: Date, calendar: Calendar.ISO, day: 2, month: 6, year: 2021}


`~D[...]` や `~T[...]` は elixir の シギル sigil である。 文字列とバイナリーのところで説明する。

In [ ]:
# date は range に使える
!

# help について
メモ $\quad$ 関数の調べ方

Helper の使い方。 help, type, info, information とか。

下のコードセルにあるように、対象のモジュールの関数名を調べ、そのヘルプを見ればけっこうくわしくわかる。

コメントアウトしてあるのは出力が大きいので、とりあえずコメントアウトして出力を抑制してある。

具体的には、Enum にあたるところにモジュール名を入れて関数のリストを出す。 Ctrl+A Ctrl+C でコピーして vscode などでペーストして読む。 調べたい関数名をヘルプの、Enum.all?/1 のところに入れて出力をコピーして、vscode などでペーストして読む


In [ ]:
# !elixir -e 'Enum.__info__(:functions) |> Enum.each(fn({function, arity}) -> IO.puts "#{function}/#{arity}" end)'

In [ ]:
# !elixir -e 'require IEx.Helpers;IEx.Helpers.h Enum.all?/1'

In [ ]:
# h 単独のドキュメントを見たい
# !elixir -e 'require IEx.Helpers;IEx.Helpers.h'

In [ ]:
# i というのもある
# !elixir -e 'x = [3,2]; require IEx.Helpers;IEx.Helpers.i x'

In [ ]:
# !elixir -e 'require IEx.Helpers;IEx.Helpers.h IO'

# enum モジュール

enum はリストなどコレクションを列挙するための一連のアルゴリズム。

* all?、any?
* chunk_every、chunk_by、map_every
* each
* map、filter、reduce
* min、max
* sort、uniq、uniq_by
* キャプチャ演算子 `(&)`


In [ ]:
# all? 関数を引数で受け取り、リストの全体が true の時、true を返す
!elixir -e 'IO.puts Enum.all?(["foo", "bar", "hello"], fn(s) -> String.length(s) == 3 end)'
!elixir -e 'IO.puts Enum.all?(["foo", "bar", "hello"], fn(s) -> String.length(s) >1 end)'

false
true


In [ ]:
# any? 少なくとも1つの要素が true と評価された場合に true を返す
!elixir -e 'IO.puts Enum.any?(["foo", "bar", "hello"], fn(s) -> String.length(s) == 5 end)'

true


In [ ]:
# chunk_every リストを小さなグループに分割する
!elixir -e 'IO.inspect Enum.chunk([1, 2, 3, 4, 5, 6], 2)'
!elixir -e 'IO.inspect Enum.chunk([1, 2, 3, 4, 5, 6], 3)'
!elixir -e 'IO.inspect Enum.chunk([1, 2, 3, 4, 5, 6], 4)'

[[1, 2], [3, 4], [5, 6]]
[[1, 2, 3], [4, 5, 6]]
[[1, 2, 3, 4]]


In [ ]:
# chunk_by 関数の戻り値が変化することによって分割する
!elixir -e 'IO.inspect Enum.chunk_by(["one", "two", "three", "four", "five"], fn(x) -> String.length(x) end)'
!elixir -e 'IO.inspect Enum.chunk_by(["one", "two", "three", "four", "five", "six"], fn(x) -> String.length(x) end)'

[["one", "two"], ["three"], ["four", "five"]]
[["one", "two"], ["three"], ["four", "five"], ["six"]]


In [ ]:
# map_every nth ごとに map 処理する
!elixir -e 'IO.inspect Enum.map_every(1..10, 3, fn x -> x + 1000 end)'
!elixir -e 'IO.inspect Enum.map_every(1..10, 1, fn x -> x + 1000 end)'
!elixir -e 'IO.inspect Enum.map_every(1..10, 0, fn x -> x + 1000 end)'

[1001, 2, 3, 1004, 5, 6, 1007, 8, 9, 1010]
[1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [ ]:
# each 新しい値を生成することなく反復する。返り値は:ok というアトム。
!elixir -e 'IO.inspect Enum.each(["one", "two", "three"], fn(s) -> IO.puts(s) end)'
!elixir -e 'IO.puts Enum.each(["one", "two", "three"], fn(s) -> IO.puts(s) end)'

one
two
three
:ok


In [ ]:
# map 関数を各要素に適用して新しいリストを生み出す
!elixir -e 'IO.inspect Enum.map([0, 1, 2, 3], fn(x) -> x - 1 end)'

[-1, 0, 1, 2]


In [ ]:
# min 最小の値を探す。 リストが空の場合エラーになる
# リストが空だったときのために予め最衣装血を生成する関数を渡すことができる
!elixir -e 'IO.inspect Enum.min([5, 3, 0, -1])'
!elixir -e 'IO.inspect Enum.min([], fn -> :foo end)'

-1
:foo


In [ ]:
# max 最大の(max/1)値を返す
!elixir -e 'IO.inspect Enum.max([5, 3, 0, -1])'
!elixir -e 'IO.inspect Enum.max([], fn -> :bar end)'

5
:bar


In [ ]:
# filter 与えられた関数によって true と評価された要素だけを得る
!elixir -e 'IO.inspect Enum.filter([1, 2, 3, 4], fn(x) -> rem(x, 2) == 0 end)'
!elixir -e 'IO.inspect Enum.filter([], fn(x) -> rem(x, 2) == 0 end)'

[2, 4]
[]


In [ ]:
# reduce リストを関数に従って単一の値へ抽出すうる。 accumulator を指定できる。
# accumulator が与えられない場合は最初の要素が用いられる。
!elixir -e 'IO.inspect Enum.reduce([1, 2, 3], 10, fn(x, acc) -> x + acc end)'
!elixir -e 'IO.inspect Enum.reduce([1, 2, 3], fn(x, acc) -> x + acc end)'
!elixir -e 'IO.inspect Enum.reduce(["a","b","c"], "1", fn(x,acc)-> x <> acc end)'

16
6
"cba1"


In [ ]:
# sort `sort/1` はソートの順序に Erlangの Term 優先順位 を使う
!elixir -e 'IO.inspect Enum.sort([5, 6, 1, 3, -1, 4])'
!elixir -e 'IO.inspect Enum.sort([:foo, "bar", Enum, -1, 4])'

# `sort/2` は、順序を決める為の関数を渡すことができる
!elixir -e 'IO.inspect Enum.sort([%{:val => 4}, %{:val => 1}], fn(x, y) -> x[:val] > y[:val] end)'

# なしの場合
!elixir -e 'IO.inspect Enum.sort([%{:count => 4}, %{:count => 1}])'

# sort/2 に :asc または :desc をソート関数として渡すことができる
!elixir -e 'IO.inspect Enum.sort([2, 3, 1], :desc)'


[-1, 1, 3, 4, 5, 6]
[-1, 4, Enum, :foo, "bar"]
[%{val: 4}, %{val: 1}]
[%{count: 1}, %{count: 4}]
[3, 2, 1]


In [ ]:
# uniq 重複した要素を取り除く
!elixir -e 'IO.inspect Enum.uniq([1, 2, 3, 2, 1, 1, 1, 1, 1])'
[1, 2, 3]

# uniq_by 重複した要素を削除するが、ユニークかどうか比較を行う関数を渡せる
!elixir -e 'IO.inspect Enum.uniq_by([%{x: 1, y: 1}, %{x: 2, y: 1}, %{x: 3, y: 3}], fn coord -> coord.y end)'

[1, 2, 3]
[%{x: 1, y: 1}, %{x: 3, y: 3}]


# キャプチャ演算子 `&` を使用した enum と無名関数 

elixir の enum モジュール内の多くの関数は、引数として無名関数を取る。

これらの無名関数は、多くの場合、キャプチャ演算子 `&` を使用して省略形で記述される。



In [ ]:
# 無名関数でのキャプチャ演算子の使用 
!elixir -e 'IO.inspect Enum.map([1,2,3], fn number -> number + 3 end)'
!elixir -e 'IO.inspect Enum.map([1,2,3], &(&1 + 3))'
!elixir -e 'plus_three = &(&1 + 3);IO.inspect Enum.map([1,2,3], plus_three)'

[4, 5, 6]
[4, 5, 6]
[4, 5, 6]


---
# パターンマッチング

パターンマッチングでは、値、データ構造、関数をマッチすることができる。

* マッチ演算子
* ピン演算子

In [ ]:
# マッチ演算子 `=` はマッチ演算子である。 マッチ演算子を通して値を代入し、
# その後、マッチさせることができる。マッチすると、方程式の結果が返され、
# 失敗すると、エラーになる
!elixir -e 'IO.puts x = 1'
!elixir -e 'x = 1;IO.puts 1 = x'
# !elixir -e 'x = 1;IO.puts 2 = x'
#=> (MatchError) no match of right hand side value: 1

1
1


In [ ]:
# リストでのマッチ演算子
!elixir -e 'IO.inspect list = [1, 2, 3]'
!elixir -e 'list = [1, 2, 3]; IO.inspect [1, 2, 3] = list'
# !elixir -e 'list = [1, 2, 3]; IO.inspect [] = list'
#=> (MatchError) no match of right hand side value: [1, 2, 3]

[1, 2, 3]
[1, 2, 3]


In [ ]:
!elixir -e 'list = [1, 2, 3]; IO.inspect [1 | tail] = list'
!elixir -e 'list = [1, 2, 3];  [1 | tail] = list; IO.inspect tail'

[1, 2, 3]
[2, 3]


In [ ]:
# タプルとマッチ演算子
!elixir -e 'IO.inspect {:ok, value} = {:ok, "Successful!"}'
!elixir -e '{:ok, value} = {:ok, "Successful!"}; IO.inspect  value'

{:ok, "Successful!"}
"Successful!"
:ok


# ピン演算子

マッチ演算子は左辺に変数が含まれている時に代入操作を行う。 

この変数を再び束縛するという挙動は望ましくない場合がある。 そうした状況のために、ピン演算子 `^` がある。

ピン演算子で変数を固定すると、新しく再束縛するのではなく既存の値とマッチする。


In [ ]:
# ピン演算子
!elixir -e 'IO.inspect x = 1'
# !elixir -e 'x = 1; IO.inspect ^x = 2'
#=> ** (MatchError) no match of right hand side value: 2
!elixir -e 'x = 1; IO.inspect {x, ^x} = {2, 1}'
!elixir -e 'x = 1;{x, ^x} = {2, 1}; IO.inspect x'
!echo
!elixir -e 'IO.inspect key = "hello"'
!elixir -e 'key = "hello"; IO.inspect %{^key => value} = %{"hello" => "world"}'
!elixir -e 'key = "hello"; IO.inspect %{^key => value} = %{"hello" => "world"}'
!elixir -e 'key = "hello"; %{^key => value} = %{"hello" => "world"}; IO.inspect value'

1
{2, 1}
2

"hello"
%{"hello" => "world"}
%{"hello" => "world"}
"world"


In [ ]:
# 関数の clause でのピン演算子
!elixir -e 'IO.inspect greeting = "Hello"'
!elixir -e 'greeting = "Hello"; IO.inspect greet = fn (^greeting, name) -> "Hi #{name}"; (greeting, name) -> "${greeting},${name}" end'
!elixir -e 'greeting = "Hello"; greet = fn (^greeting, name) -> "Hi #{name}"; (greeting, name) -> "#{greeting},#{name}" end; IO.inspect greet.("Hello","Sean")'
!elixir -e 'greeting = "Hello"; greet = fn (^greeting, name) -> "Hi #{name}"; (greeting, name) -> "#{greeting},#{name}" end; IO.inspect greet.("Mornin","Sean")'

"Hello"
#Function<12.99386804/2 in :erl_eval.expr/5>
"Hi Sean"
"Mornin,Sean"


# いまここ

# 制御構造 control structure

* if と unless
* case
* cond
* with

if と unless 

elixir の if と unless は ruby と同じ。

elixir は if と unless はマクロとして定義されている。

この実装は kernel module で知ることができる。

elixir では偽とみなされる値は nil と真理値の false だけだということに留意。

In [ ]:
iex> if String.valid?("Hello") do
...>   "Valid string!"
...> else
...>   "Invalid string."
...> end
"Valid string!"

iex> if "a string value" do
...>   "Truthy"
...> end
"Truthy"
unless/2 は if/2 のように使いますが、条件が否定される時だけ作用します:

Copy
iex> unless is_integer("hello") do
...>   "Not an Int"
...> end
"Not an Int"
case 
複数のパターンに対してマッチする必要があるなら、 case/2 を使うことができます:

Copy
iex> case {:ok, "Hello World"} do
...>   {:ok, result} -> result
...>   {:error} -> "Uh oh!"
...>   _ -> "Catch all"
...> end
"Hello World"
_ 変数は case/2 命令文の中に含まれる重要な要素です。これが無いと、マッチするものが見あたらない場合にエラーが発生します:

Copy
iex> case :even do
...>   :odd -> "Odd"
...> end
** (CaseClauseError) no case clause matching: :even

iex> case :even do
...>   :odd -> "Odd"
...>   _ -> "Not Odd"
...> end
"Not Odd"
_ を”他の全て”にマッチする else と考えましょう。

case/2 はパターンマッチングに依存しているため、パターンマッチングと同じルールや制限が全て適用されます。既存の変数に対してマッチさせようという場合にはピン ^ 演算子を使わなくてはいけません:

Copy
iex> pie = 3.14
3.14
iex> case "cherry pie" do
...>   ^pie -> "Not so tasty"
...>   pie -> "I bet #{pie} is tasty"
...> end
"I bet cherry pie is tasty"
case/2 のもう1つの素晴らしい特徴として、ガード節に対応していることがあげられます:

この例は公式のElixirのGetting Startedガイドから直接持ってきています。

Copy
iex> case {1, 2, 3} do
...>   {1, x, 3} when x > 0 ->
...>     "Will match"
...>   _ ->
...>     "Won't match"
...> end
"Will match"
公式ドキュメントからExpressions allowed in guard clausesを読んでみてください。

cond 
値ではなく、条件をマッチさせる必要がある時には、 cond/1 を使うことができます。これは他の言語でいうところの else if や elsif のようなものです:

この例は公式のElixirのGetting Startedガイドから直接持ってきています。

Copy
iex> cond do
...>   2 + 2 == 5 ->
...>     "This will not be true"
...>   2 * 2 == 3 ->
...>     "Nor this"
...>   1 + 1 == 2 ->
...>     "But this will"
...> end
"But this will"
case のように、 cond はマッチしない場合にエラーを発生させます。これに対処するには、 true になる条件を定義すればよいです:

Copy
iex> cond do
...>   7 + 1 == 0 -> "Incorrect"
...>   true -> "Catch all"
...> end
"Catch all"
with 
特殊形式の with/1 はネストされた case/2 文を使うような時やきれいにパイプできない状況に便利です。 with/1 式はキーワード, ジェネレータ, そして式から成り立っています。

ジェネレータについてはリスト内包表記のレッスンでより詳しく述べますが、今は <- の右側と左側を比べるのにパターンマッチングが使われることを知っておくだけでよいです。

with/1 の簡単な例から始め、その後さらなる例を見てみましょう:

Copy
iex> user = %{first: "Sean", last: "Callan"}
%{first: "Sean", last: "Callan"}
iex> with {:ok, first} <- Map.fetch(user, :first),
...>      {:ok, last} <- Map.fetch(user, :last),
...>      do: last <> ", " <> first
"Callan, Sean"
式がマッチに失敗した場合はマッチしない値が返されます:

Copy
iex> user = %{first: "doomspork"}
%{first: "doomspork"}
iex> with {:ok, first} <- Map.fetch(user, :first),
...>      {:ok, last} <- Map.fetch(user, :last),
...>      do: last <> ", " <> first
:error
それでは、 with/1 を使わない長めの例と、それをどのようにリファクタリングできるかを見てみましょう:

Copy
case Repo.insert(changeset) do
  {:ok, user} ->
    case Guardian.encode_and_sign(user, :token, claims) do
      {:ok, jwt, full_claims} ->
        important_stuff(jwt, full_claims)

      error ->
        error
    end

  error ->
    error
end
with/1 を導入するとコードが短く、わかりやすくなります:

Copy
with {:ok, user} <- Repo.insert(changeset),
     {:ok, jwt, full_claims} <- Guardian.encode_and_sign(user, :token, claims),
     do: important_stuff(jwt, full_claims)
Elixir 1.3からは with/1 で else を使えます:

Copy
import Integer

m = %{a: 1, c: 3}

a =
  with {:ok, number} <- Map.fetch(m, :a),
    true <- is_even(number) do
      IO.puts "#{number} divided by 2 is #{div(number, 2)}"
      :even
  else
    :error ->
      IO.puts("We don't have this item in map")
      :error

    _ ->
      IO.puts("It is odd")
      :odd
  end
これは case のようなパターンマッチングを提供することで、エラーを扱いやすくします。渡されるのはマッチングに失敗した最初の表現式の値です。

Caught a mistake or want to contribute to the lesson? Edit this page on GitHub!
← パターンマッチング
関数 →
bg bn de en es fr gr id it ja ko ms no pl pt ru sk ta th tr uk vi zh-hans zh-hant
Elixir 1.10.1 - Erlang/OTP 22.0 [erts-10.5.3]
Menu
プログラミング言語Elixirのレッスン
© 2021 Sean Callan All rights reserved.

Toggle